In [4]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn.model_selection as ms


In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
total=pd.concat([train,test])
print str(len(train))+":"+str(len(test))
total.info()

891:418
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [6]:
total.isnull().sum()

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

In [7]:
#train[['Sex','Survived']].groupby(['Sex'], as_index=False).mean()
total['FamilySize']=total['SibSp']+total['Parch']+1
#total[['FamilySize','Survived']].groupby(['FamilySize'], as_index=False).mean()

In [8]:
total['IsAlone']=1
total.loc[total['FamilySize']>1, 'IsAlone']=0
#total[['IsAlone','Survived']].groupby(['IsAlone'], as_index=False).mean()

In [9]:
total['Embarked'].fillna('S',inplace=True)
total[['Embarked','Survived']].groupby('Embarked', as_index=False).mean()

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [10]:
total['Fare'].fillna(total['Fare'].median(),inplace=True)
total['FareCategorical']=pd.qcut(total['Fare'],4)
total[['FareCategorical','Survived']].groupby('FareCategorical', as_index=False).mean()

,FareCategorical,Survived
0,"(-0.001, 7.896]",0.197309
1,"(7.896, 14.454]",0.303571
2,"(14.454, 31.275]",0.441048
3,"(31.275, 512.329]",0.600000


In [11]:
mu=total['Age'].mean()
delta=total['Age'].std()
rand_list=np.random.randint(mu-delta,mu+delta,size=total['Age'].isnull().sum())
total.loc[total['Age'].isnull(), 'Age']=rand_list
total['Age']=total['Age'].astype(int)
total['AgeCategorical']=pd.cut(total['Age'],5)
total[['AgeCategorical','Survived']].groupby('AgeCategorical', as_index=False).mean()


,AgeCategorical,Survived
0,"(-0.08, 16.0]",0.536364
1,"(16.0, 32.0]",0.344749
2,"(32.0, 48.0]",0.384030
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


In [12]:
total.drop('PassengerId',axis=1,inplace=True)
total.drop('Name',axis=1,inplace=True)
total.drop('Cabin',axis=1,inplace=True)
total.drop('SibSp',axis=1,inplace=True)
total.drop('Parch',axis=1,inplace=True)
total.drop('Ticket',axis=1,inplace=True)
total.drop('FareCategorical',axis=1,inplace=True)
total.drop('AgeCategorical',axis=1,inplace=True)


age_bins=[-1,16,32,48,64,80]
total['Age'] = pd.cut(total['Age'],age_bins , labels=range(len(age_bins)-1)) 

fare_bins=[-1,7.896,14.454,31.275,512.3292]
total['Fare']=pd.cut(total['Fare'], fare_bins, labels=range(len(fare_bins)-1))

total['Embarked']=total['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2 }).astype(int)
total['Sex']=total['Sex'].map( { 'female' : 0 , 'male' : 1 } ).astype(int)





In [13]:
train_y = train['Survived']

total.drop('Survived', axis=1, inplace=True)
total_array = total.as_matrix()

train_x = total_array[:len(train_y),:]
test_x = total_array[len(train_y):,:]




#predictions = gbm.predict(test_x)
#submission = pd.DataFrame({ 'PassengerId': test['PassengerId'],
#                            'Survived': predictions })
#submission.to_csv("submission.csv", index=False)


In [ ]:
#param_grid = {'max_depth':range(3,10,1), 'min_child_weight':range(1,6,1)} #3,4
#param_grid = {'gamma':[i/10.0 for i in range(0,7)]}
param_grid = {'subsample':[i/100.0 for i in range(60,100,5)],
               'colsample_bytree':[i/100.0 for i in range(60,100,5)]}

model = xgb.XGBClassifier(learning_rate =0.1, 
                      n_estimators=140, 
                      max_depth=3,
                      min_child_weight=4, 
                      gamma=0.6, 
                      subsample=0.8, 
                      colsample_bytree=0.8,
                      objective= 'binary:logistic', 
                      n_jobs=4, 
                      scale_pos_weight=1)

grid = ms.GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', n_jobs=-1, verbose=1)
grid.fit(train_x,train_y)




Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.4min


In [ ]:
print (grid.best_score_)
print (grid.best_params_)